### Fuentes OCID

In [0]:
%sql
select count(distinct ocid) N
from planning_hist
where budget_amount is null


select ocid,count(*) N
from planning_hist
group by ocid
order by count(*) desc

select *
from planning_hist
where 1=1
and ocid ='ocds-5wno2w-LS-GADMLA-2021-001-91040'
--and ocid='ocds-5wno2w-R2-SIE-MIMG-098-2021-44856'

In [0]:
%sql
select anio,count(*)
from tender_hist
group by anio


select ocid,count(*),sum(numberOfTenderers) tends
from tender_hist
group by ocid 
order by count(*) desc


select *
from tender_hist
where 1=1
--and ocid ='ocds-5wno2w-LS-GADMLA-2021-001-91040'
--and ocid='ocds-5wno2w-R2-SIE-MIMG-098-2021-44856'
and tenderPeriod_durationInDays is null 

In [0]:
%sql
select ocid,count(*),
count(distinct description) descripts,
max(date),min(date),
date_diff(
  cast(max(date) as timestamp),
  cast(min(date) as timestamp)
  ),
count(distinct contractPeriod_durationInDays),
count(distinct status) statuses
from award_hist
group by ocid
order by count(*) desc


select *
from award_hist
where 1=1
--and ocid ='ocds-5wno2w-LS-GADMLA-2021-001-91040'
and ocid='ocds-5wno2w-CDC-EERSSA-064-2020-3240'

In [0]:
%sql
select count(distinct ocid) N
from contract_hist
group by ocid
order by count(*) desc



select *
from contract_hist
where 1=1
--and ocid ='ocds-5wno2w-LS-GADMLA-2021-001-91040'
and ocid='ocds-5wno2w-CDC-EERSSA-064-2020-3240'

### Integración

In [0]:
select *
from planning_hist
where ocid like '%ocds-5wno2w-CE-20200001758187%'


select count(distinct ocid)
from planning_hist;

select count(distinct ocid)
from tender_hist;


select *
from tender_hist;
where 

select count(distinct ocid)
from award_hist;

select count(distinct ocid)
from contract_hist;

select *
from planning_hist pl
full outer join contract_hist ct 
on pl.ocid=ct.ocid and pl.anio=ct.anio
where pl.ocid is null 

s


In [0]:

create or replace temporary view fuente_planning as
select *
from planning_hist
--where anio=2021
;



create or replace temporary view fuente_tender as
select *,
case 
  when tenderPeriod_endDate is null then enquiryPeriod_endDate 
  else null end as tenderPeriod_endDate2,
case 
  when mainProcurementCategory is null and procurementMethodDetails like '%SUMINISTROS DE%' then 'goods'
  when mainProcurementCategory is null and procurementMethodDetails like '%ADQUISICION DE%' then 'goods'
  when mainProcurementCategory is null and procurementMethodDetails like '%ADQUISICIÓN DE%' then 'goods'
  when mainProcurementCategory is null and procurementMethodDetails like '%adquisición de%' then 'goods'
  when mainProcurementCategory is null and procurementMethodDetails like '%PRODUCTOS DE%' then 'goods'
  when mainProcurementCategory is null and procurementMethodDetails like '%PROVISIÓN DE%' then 'goods'
  when mainProcurementCategory is null and procurementMethodDetails like '%PROVISION DE%' then 'goods'
  when mainProcurementCategory is null and procurementMethodDetails like '%SERVICIO DE%' then 'services'
  when mainProcurementCategory is null and procurementMethodDetails like '%SERVICIOS DE%' then 'services'
  when mainProcurementCategory is null and procurementMethodDetails like '%SERVICIO INTEGRAL%' then 'services'
  when mainProcurementCategory is null and procurementMethodDetails like '%PRODUCCION DE%' then 'works'
  else mainProcurementCategory
end as mainProcurementCategory2,
case 
  when hasEnquiries is null and procurementMethod='direct' then 0
  when hasEnquiries is null and procurementMethod='selective' then 0
  else hasEnquiries
end as hasEnquiries2
from tender_hist 
--where anio=2021
;




create or replace temporary view fuente_award as
select 
coalesce(aw.ocid,max(ct.ocid)) ocid,
max(aw.description) description_award,
max(aw.status) status_award,
max(aw.date) date_award,
sum(aw.amount) total_amount_award,
coalesce(
  max(aw.contractPeriod_startDate),
  max(ct.contractPeriod_startDate)) contractPeriod_startDate,
coalesce(
  max(aw.contractPeriod_endDate),
  max(ct.contractPeriod_endDate)) contractPeriod_endDate,
coalesce(
  avg(aw.contractPeriod_durationInDays),
  avg(ct.contractPeriod_durationInDays)) contractPeriod_durationInDays,
coalesce(max(aw.anio),max(ct.anio)) anio,
count(*) num_adjudicaciones,
max(ct.status) status_contract,
coalesce(sum(ct.amount),sum(aw.amount)) total_amount_contract,
max(dateSigned) dateSigned
from award_hist aw
full outer join contract_hist ct
on aw.ocid=ct.ocid and aw.id=ct.awardID and aw.anio=ct.anio
where 1=1
and ct.ocid is not null
--where aw.anio=2021
group by aw.ocid;


In [0]:

select *
from fuente_award
where ocid='ocds-5wno2w-LS-GADMLA-2021-001-91040'


select count(distinct ocid)
from fuente_award

select status_award,count(*)
from fuente_award
group by status_award

In [0]:
create or replace temporary view procesos_integrados as
select 
coalesce(coalesce(pl.ocid,td.ocid),aw.ocid) ocid,
coalesce(pl.anio,aw.anio) anio,
coalesce(pl.rationale,td.description) rationale,
coalesce(pl.budget_amount,td.value_amount) budget_amount,
td.status,
td.procuringEntity_id,
td.procurementMethod,td.procurementMethodDetails,
td.mainProcurementCategory2 mainProcurementCategory,
td.awardCriteria,
td.tenderPeriod_startDate,
td.tenderPeriod_endDate2 tenderPeriod_endDate,
td.tenderPeriod_maxExtentDate,
coalesce(
  td.tenderPeriod_durationInDays,
  datediff(td.tenderPeriod_endDate2,td.tenderPeriod_startDate)
) tenderPeriod_durationInDays,
td.enquiryPeriod_startDate,
td.enquiryPeriod_endDate,
td.enquiryPeriod_maxExtentDate,
coalesce(
  td.enquiryPeriod_durationInDays,
  datediff(td.enquiryPeriod_endDate,td.enquiryPeriod_startDate)
  ) enquiryPeriod_durationInDays,
td.hasEnquiries2 hasEnquiries,
td.eligibilityCriteria,
td.awardPeriod_startDate,
td.awardPeriod_endDate,td.awardPeriod_maxExtentDate,
td.awardPeriod_durationInDays,td.numberOfTenderers,
aw.description_award,
aw.status_award,
aw.date_award,aw.total_amount_award,
aw.contractPeriod_startDate,
aw.contractPeriod_endDate,
aw.contractPeriod_durationInDays,
aw.num_adjudicaciones,
aw.status_contract,
aw.total_amount_contract,
aw.dateSigned,
case when pl.ocid is null then 1 else 0 end as en_planificacion
from fuente_planning pl
full outer join fuente_tender td
on pl.ocid=td.ocid and pl.anio=td.anio
full outer join fuente_award aw
on td.ocid=aw.ocid and td.anio=aw.anio
where 1=1
and aw.ocid is not null
--where pl.ocid ='ocds-5wno2w-LS-GADMLA-2021-001-91040'

In [0]:
select 
tenderPeriod_startDate,
tenderPeriod_endDate,
--tenderPeriod_maxExtentDate, --es igual a tenderPeriod_endDate
tenderPeriod_durationInDays,
enquiryPeriod_startDate,
enquiryPeriod_endDate,
--enquiryPeriod_maxExtentDate, --es igual a enquiryPeriod_endDate
enquiryPeriod_durationInDays,
awardPeriod_startDate,
awardPeriod_endDate,
--awardPeriod_maxExtentDate, --es igual a awardPeriod_endDate
awardPeriod_durationInDays,
contractPeriod_startDate,
contractPeriod_endDate,
contractPeriod_durationInDays,
status_contract,
*
from procesos_integrados
--where ocid ='ocds-5wno2w-LS-GADMLA-2021-001-91040'
--where budget_amount=value_amount is null
--where rationale is null
--where tenderPeriod_endDate is null
where tenderPeriod_startDate is null

select *
from procesos_integrados
where 1=1
--and procurementMethod='direct'
and procurementMethodDetails='Contratacion directa'


select procurementMethod,count(*),
sum(case when hasEnquiries is null then 1 else 0 end) a
from procesos_integrados
group by procurementMethod

select distinct en_planificacion
from procesos_integrados
where 1=1
--and ocid='ocds-5wno2w-LS-GADMLA-2021-001-91040'
and awardCriteria  is null


select tenderPeriod_durationInDays,count(*) N
from procesos_integrados
group by tenderPeriod_durationInDays

select distinct awardPeriod_durationInDays
from procesos_integrados
where anio=2020
--and procurementMethod='direct'
and mainProcurementCategory='goods'

### Limpieza

In [0]:
%python
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
# Configurar Pandas para evitar notación científica
pd.options.display.float_format = '{:,.2f}'.format  # Dos decimales, con separador de miles

In [0]:
%py

df=spark.sql("select * from procesos_integrados").toPandas()

df.loc[df["description_award"]=='nan',"description_award"]=np.nan

In [0]:
%python
print(df.columns)
print(df.shape)


In [0]:
%python
df[["ocid","rationale","budget_amount","status","procurementMethod",
    "procurementMethodDetails","awardCriteria","total_amount_award","num_adjudicaciones"]].describe(include='all')


In [0]:
%python
df.dtypes

In [0]:
%python
# Análisis de valores nulos
def analizar_nulos(df):
    print("Resumen de valores nulos:")
    nulos = df.isnull().sum()
    porcentaje = (nulos / len(df)) * 100
    resumen = pd.DataFrame({'Total Nulos': nulos, 'Porcentaje (%)': porcentaje})
    print(resumen)
    
    # Visualizar con heatmap
    # plt.figure(figsize=(10,6))
    # sns.heatmap(df.isnull(), cmap='viridis', cbar=False, yticklabels=False)
    # plt.title("Mapa de valores nulos")
    # plt.show()

# Llamar a la función
analizar_nulos(df)

In [0]:
%python
# Calcular el número de valores nulos por fila
df['Nulos por Fila'] = df.isnull().sum(axis=1)

# Calcular el porcentaje de columnas faltantes por fila
df['% Nulos'] = (df['Nulos por Fila'] / df.shape[1]) * 100

# Ordenar el DataFrame por el porcentaje de nulos en orden descendente
df_ordenado = df.sort_values(by='% Nulos', ascending=False)

# Mostrar el resultado
print(df_ordenado[['Nulos por Fila', '% Nulos']])

In [0]:
%python
(df_ordenado["Nulos por Fila"]<17).sum()

#### Imputar datos

In [0]:
%python
for col in ['tenderPeriod_startDate', 'tenderPeriod_endDate', 
            'enquiryPeriod_startDate', 'enquiryPeriod_endDate',
            'awardPeriod_startDate', 'awardPeriod_endDate',
            'contractPeriod_startDate','contractPeriod_endDate']:
    df[col] = pd.to_datetime(df[col], errors='coerce')


# Paso 1: Calcular la duración donde sea posible
df['tenderPeriod_durationInDays'] = np.where(
    df['tenderPeriod_durationInDays'].isna() & df['tenderPeriod_startDate'].notna() & df['tenderPeriod_endDate'].notna(),
    (df['tenderPeriod_endDate'] - df['tenderPeriod_startDate']).dt.days,
    df['tenderPeriod_durationInDays']
)


# Paso 1: Calcular la duración donde sea posible
df['enquiryPeriod_durationInDays'] = np.where(
    df['tenderPeriod_durationInDays'].isna() & df['enquiryPeriod_startDate'].notna() & df['enquiryPeriod_endDate'].notna(),
    (df['enquiryPeriod_endDate'] - df['enquiryPeriod_startDate']).dt.days,
    df['enquiryPeriod_durationInDays']
)


# Paso 1: Calcular la duración donde sea posible
df['awardPeriod_durationInDays'] = np.where(
    df['tenderPeriod_durationInDays'].isna() & df['awardPeriod_startDate'].notna() & df['awardPeriod_endDate'].notna(),
    (df['awardPeriod_endDate'] - df['awardPeriod_startDate']).dt.days,
    df['awardPeriod_durationInDays']
)


# Paso 1: Calcular la duración donde sea posible
df['contractPeriod_durationInDays'] = np.where(
    df['tenderPeriod_durationInDays'].isna() & df['contractPeriod_startDate'].notna() & df['contractPeriod_endDate'].notna(),
    (df['contractPeriod_endDate'] - df['contractPeriod_startDate']).dt.days,
    df['contractPeriod_durationInDays']
)

In [0]:
%python

# Imputación basada en relaciones entre períodos (mediana agrupada)
grouped_median = df.groupby([
    'anio',
    'status',
    'mainProcurementCategory'
])[['tenderPeriod_durationInDays', 'enquiryPeriod_durationInDays', 'awardPeriod_durationInDays','contractPeriod_durationInDays']].median()

# Reseteamos el índice para usar merge en lugar de iterrows
df_grupos = grouped_median.reset_index()

# Optimización 1: Usamos merge en lugar de iterrows
df = df.merge(df_grupos, on=['anio', 'status', 'mainProcurementCategory'], suffixes=('', '_median'), how='left')

# Optimización 2: Reemplazamos valores NaN con los valores medianos escalados, usando operaciones vectorizadas
df['tenderPeriod_durationInDays'] = np.where(
    df['tenderPeriod_durationInDays'].isna(),
    np.ceil(df['tenderPeriod_durationInDays_median']),
    df['awardPeriod_durationInDays']
)


df['enquiryPeriod_durationInDays'] = np.where(
    df['enquiryPeriod_durationInDays'].isna(),
    np.ceil(df['enquiryPeriod_durationInDays_median'] * 0.2),
    df['enquiryPeriod_durationInDays']
)

df['awardPeriod_durationInDays'] = np.where(
    df['awardPeriod_durationInDays'].isna(),
    np.ceil(df['enquiryPeriod_durationInDays_median'] * 0.3),
    df['awardPeriod_durationInDays']
)


df['contractPeriod_durationInDays'] = np.where(
    df['awardPeriod_durationInDays'].isna(),
    np.ceil(df['contractPeriod_durationInDays_median']),
    df['contractPeriod_durationInDays']
)


# Eliminamos las columnas auxiliares usadas en merge
df.drop(columns=['tenderPeriod_durationInDays_median', 'enquiryPeriod_durationInDays_median', 'awardPeriod_durationInDays_median','contractPeriod_durationInDays_median'], inplace=True)


In [0]:
%python

#imputación basada en relaciones entre períodos (mediana agrupada)
grouped_median_budget = df.groupby([
    'anio',
    'status',
    'mainProcurementCategory'
])[['budget_amount']].median()

# Reseteamos el índice para usar merge en lugar de iterrows
df_grupos_budget = grouped_median_budget.reset_index()

# Optimización 1: Usamos merge en lugar de iterrows
df = df.merge(df_grupos_budget, on=['anio', 'status', 'mainProcurementCategory'], suffixes=('', '_median'), how='left')

# Optimización 2: Reemplazamos valores NaN con los valores medianos escalados, usando operaciones vectorizadas
df['budget_amount'] = np.where(
    df['budget_amount'].isna(),
    np.ceil(df['budget_amount_median']),
    df['budget_amount']
)


# Eliminamos las columnas auxiliares usadas en merge
df.drop(columns=['budget_amount_median'], inplace=True)

In [0]:
%python
display(df)

### Variables Texto 

In [0]:
%python
!pip install spacy

import re
import spacy
import spacy.cli
from scipy import sparse
spacy.cli.download("es_core_news_sm")
nlp = spacy.load("es_core_news_sm")  # Modelo en español

In [0]:
%python
def limpiar_texto(texto):
    if pd.isna(texto): return ""
    texto = texto.lower()
    texto = re.sub(r'\d+', '', texto)  # Eliminar números
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar puntuación
    doc = nlp(texto)
    texto = " ".join([token.lemma_ for token in doc if not token.is_stop])  # Lematización sin stopwords
    return texto

In [0]:
%python
#se limpia el texto de la columna de justificacion de adjudicacion

justificacion_plan=pd.DataFrame(df['rationale'].unique(),columns=["rationale"])
justificacion_plan['rationale_clean'] = justificacion_plan['rationale'].apply(limpiar_texto)
display(justificacion_plan)


In [0]:
%python
df=pd.merge(df,justificacion_plan,left_on='rationale',right_on='rationale')


In [0]:
%python
#guardar como parquet

import pyarrow

df.to_parquet('/dbfs/mnt/aci-layer1/MODELOS_ANALITICOS/ocid_data_total.parquet', index=False)

In [0]:
%run ../../../GENERICS/PY_DataLakeFunctions

In [0]:
%python
#cargar df nuevamente desde parquet guardado 
#df=spark.read.parquet('/mnt/aci-layer1/MODELOS_ANALITICOS/ocid_data_total.parquet').toPandas()

In [0]:
%python
#se vectoriza la columna de justificacion de planificacion de compra publica

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500)  # Se limita a las 500 palabras más relevantes
X_tfidf = vectorizer.fit_transform(df['rationale_clean'])

In [0]:
%python
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distiluse-base-multilingual-cased')
X_embed = model.encode(df['rationale_clean'])

In [0]:
%python
print(type(X_tfidf))
print(X_tfidf.shape)

In [0]:
%python
from sklearn.cluster import KMeans

num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_model['cluster_rationale'] = kmeans.fit_predict(X_tfidf)


In [0]:
%python
df[['cluster_rationale','rationale']].unique()

In [0]:
%python
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5, random_state=42)
topics = lda.fit_transform(X_tfidf)
df['topic_rationale'] = topics.argmax(axis=1)


In [0]:
%python
lda.transform(X_tfidf[-2:])

In [0]:
%python

df[['topic_rationale','rationale']]

In [0]:
%python
df[["anio","budget_amount","tenderPeriod_durationInDays","enquiryPeriod_durationInDays",
          "hasEnquiries","awardPeriod_durationInDays","total_amount_award","num_adjudicaciones"]].dropna().groupby("tenderPeriod_durationInDays")["tenderPeriod_durationInDays"].count()

### Modelamiento

In [0]:
%python
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

In [0]:
%python
!pip install tensorflow
!pip install scikit-optimize

In [0]:
%python
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

from skopt import gp_minimize
from skopt.space import Integer, Real

In [0]:
%python
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Integer

#### Organizar datos

In [0]:
%python
#dataframe para el modelamiento
#se cambia ocid a indice
df_model=df.copy()

df_model=df_model.set_index("ocid")

In [0]:
%python
display(df_model.head(10))

In [0]:
%python
cols_num=["anio","budget_amount","tenderPeriod_durationInDays","enquiryPeriod_durationInDays"
          ,
          "hasEnquiries","awardPeriod_durationInDays","total_amount_award","num_adjudicaciones"]

cols_cat=["status","procurementMethod","mainProcurementCategory"
          ,"rationale"
          ]


for col in cols_cat:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])


In [0]:
%python
df_model_total=df_model[cols_num+cols_cat].dropna()
df_model_total.shape

In [0]:
%python
#df_model_total.to_parquet('/dbfs/mnt/aci-layer1/MODELOS_ANALITICOS/ocid_data_total.parquet', index=False)
df_model_total=spark.read.parquet('/mnt/aci-layer1/MODELOS_ANALITICOS/ocid_data_total.parquet').toPandas()

In [0]:
%python
analizar_nulos(df_model_total)

In [0]:
%python
# Calcular el número de valores nulos por fila
df_model_total['Nulos por Fila'] = df_model_total.isnull().sum(axis=1)

# Calcular el porcentaje de columnas faltantes por fila
df_model_total['% Nulos'] = (df_model_total['Nulos por Fila'] / df_model_total.shape[1]) * 100

# Ordenar el DataFrame por el porcentaje de nulos en orden descendente
df_ordenado = df_model_total.sort_values(by='% Nulos', ascending=False)

# Mostrar el resultado
print(df_ordenado[['Nulos por Fila', '% Nulos']])

In [0]:
%python
(df_ordenado["Nulos por Fila"]<=1).sum()

In [0]:
%python
display(df_model_total)

#### Preprocesamiento

In [0]:
%python
df_ml=df_model_total.sort_values(by=["anio"])  # Ordenar por año

#df_ml=df_model_total[df_model_total.anio==2024]

df_ml.shape

In [0]:
%python
df_ml.head(5)

In [0]:
%python
#df_ml.groupby("awardPeriod_durationInDays")["awardPeriod_durationInDays"].count()
df_ml["hasEnquiries"].unique()

In [0]:
%python
# Seleccionar características numéricas para modelar
features = [col for col in df_ml.columns if col not in ["anio"]]
#scaler =  StandardScaler()
scaler =  MinMaxScaler()
df_ml[features] = scaler.fit_transform(df_ml[features])

In [0]:
%python
df_ml.describe()

In [0]:
%python
df_ml["tenderPeriod_durationInDays"].unique()

In [0]:
%python
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test = train_test_split(df_ml[features], test_size=0.3, shuffle=False)


#### Autoencoder

In [0]:
%python
#!pip install tensorflow
!pip install scikeras

In [0]:
%python
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import ParameterGrid
from scikeras.wrappers import KerasRegressor
from functools import partial
from keras.layers import Input, Dense
from keras.models import Model

In [0]:
%python
class CustomKerasRegressor(KerasRegressor):
    def __init__(self, **kwargs):
        self.encoding_dim = kwargs.pop('encoding_dim', 32)
        super().__init__(**kwargs)

    def get_params(self, deep=True):
        params = super().get_params(deep)
        params.update({'encoding_dim': self.encoding_dim})
        return params

    def set_params(self, **params):
        self.encoding_dim = params.pop('encoding_dim', self.encoding_dim)
        return super().set_params(**params)

def build_autoencoder(encoding_dim=32, input_dim=X_train.shape[1]):
    """Construye un autoencoder con la dimensión de codificación especificada"""
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')

    return autoencoder

def create_model(encoding_dim=32):
    return build_autoencoder(encoding_dim=encoding_dim, input_dim=X_train.shape[1])

autoencoder = CustomKerasRegressor(
    build_fn=create_model,
    verbose=0
)

In [0]:
%python
search = BayesSearchCV(
    estimator=autoencoder,
    search_spaces={
        'encoding_dim': Integer(3,10),  # Tamaño del espacio latente
        'batch_size': Integer(32,128),
        'epochs': Integer(50,100)
    },
    n_iter=20,  
    random_state=42
)

# Asegúrate de que X_train esté definido y contenga tus datos de entrenamiento
search.fit(X_train, X_train)
best_params_bayes = search.best_params_
print("Mejores hiperparámetros (Búsqueda Bayesiana):", best_params_bayes)

In [0]:
%python
# Entrenar el autoencoder con los mejores parámetros encontrados
best_autoencoder = build_autoencoder(
  10 #best_params_bayes['encoding_dim']
  , X_train.shape[1])
best_autoencoder.fit(X_train, X_train, 
                     epochs=  100,#best_params_bayes['epochs'], 
                     batch_size= 32, ##best_params_bayes['batch_size'], 
                     verbose=1)

In [0]:
%python
# Calcular la reconstrucción de cada dato
X_reconstruido_autoencoder = best_autoencoder.predict(df_ml[features])
reconstruction_error = np.mean(np.abs(df_ml[features] - X_reconstruido_autoencoder), axis=1)

# Añadir el error al dataframe original
df_model_total['reconstruction_error_autoencoder'] = np.nan
df_model_total.loc[df_ml.index, 'reconstruction_error_autoencoder'] = reconstruction_error

In [0]:
%python
display(df_model_total)

#### K means

In [0]:
%python
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Integer

In [0]:
%python
# Determinar el número óptimo de clusters con el método del codo
inertia = []
K_range = range(2, 15)  # Probar con diferentes valores de k

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_ml[cols_cat+cols_num])
    inertia.append(kmeans.inertia_)


# Graficar la inercia para visualizar el "codo"
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia, marker="o")
plt.xlabel("Número de Clusters (k)")
plt.ylabel("Inercia")
plt.title("Método del Codo para K-Means")
plt.show()

In [0]:
%python
# Selección automática de k con la mejor métrica de silhouette
best_k = max(K_range, key=lambda k: silhouette_score(df_ml, KMeans(n_clusters=k, random_state=42, n_init=10).fit_predict(df_ml)))

print(f"Mejor número de clusters según silhouette score: {best_k}")

In [0]:
%python
# Búsqueda de hiperparámetros con Optimización Bayesiana
def objective(params):
    k = params[0]
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(df_ml[cols_cat+cols_num])
    return -silhouette_score(df_ml, labels)  # Queremos maximizar el score, por eso el signo negativo

space = [Integer(2, 15, name="k")]

res = gp_minimize(objective, space, n_calls=10, random_state=42)

In [0]:
%python
res.x[0]

In [0]:
%python
# Entrenar el mejor modelo
best_k = 12 #res.x[0]
final_kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
df_ml["cluster_k_mean"] = final_kmeans.fit_predict(df_ml[cols_num+cols_cat])


In [0]:
%python
display(df_ml.groupby("cluster_k_mean")["cluster_k_mean"].count())

In [0]:
%python
# Calcular la distancia de cada punto a su centroide más cercano
distances = np.min(final_kmeans.transform(df_ml[cols_num+cols_cat]), axis=1)
df_ml["anomaly_score_kmean"] = distances  # Cuanto mayor la distancia, más anómalo es el registro

# Guardar los resultados en df_model_total
df_model_total["cluster_k_mean"] = df_ml["cluster_k_mean"]
df_model_total["anomaly_score_k_mean"] = df_ml["anomaly_score_kmean"]

# Mostrar los registros más sospechosos
df_model_total.sort_values(by="anomaly_score_k_mean", ascending=False).head(10)

#### Isolation Forest

In [0]:
%python
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from skopt import BayesSearchCV
from skopt.space import Real, Integer

In [0]:
%python
# Grid Search para optimizar hiperparámetros
# -----------------------------------------
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles
    'max_samples': ['auto', 0.8, 0.9],  # Proporción de muestras usadas
    'contamination': [0.05, 0.1, 0.15],  # Proporción estimada de anomalías
    'max_features': [0.5, 0.75, 1.0],  # Fracción de características usadas
}


best_score = float('-inf')
best_params = None

for params in ParameterGrid(param_grid):
    model = IsolationForest(**params, random_state=42)
    model.fit(df_ml[cols_num+cols_cat])
    scores = model.decision_function(df_ml[cols_num+cols_cat]).mean()  # Evaluar media de puntajes
    if scores > best_score:
        best_score = scores
        best_params = params

print("Mejores hiperparámetros (Grid Search):", best_params)


In [0]:
%python
# Definir una función de evaluación personalizada para BayesSearchCV
# -----------------------------------------
def anomaly_score(estimator, X, y=None):
    """ Usa la media del decision_function como métrica de evaluación. """
    scores = estimator.decision_function(X)
    return np.mean(scores)



# Búsqueda Bayesiana para optimización avanzada
# -----------------------------------------
bayes_search = BayesSearchCV(
    IsolationForest(random_state=42),
    {
        'n_estimators': Integer(50, 300),
        'max_samples': Real(0.5, 1.0),
        'contamination': Real(0.01, 0.2),
        'max_features': Real(0.5, 1.0),
    },
    n_iter=30,  
    random_state=42,
    scoring=anomaly_score,
    cv=3  
)

bayes_search.fit(df_ml[cols_num+cols_cat])
best_params_bayes = bayes_search.best_params_
print("Mejores hiperparámetros (Búsqueda Bayesiana):", best_params_bayes)

In [0]:
%python
# Entrenar el modelo final con los mejores parámetros
# -----------------------------------------
#best_model_iso = IsolationForest(**best_params_bayes, random_state=42)
best_model_iso = IsolationForest(
  n_estimators=50,
  contamination=0.01,
  max_features=1.0,
  max_samples=0.5,
  random_state=42)
best_model_iso.fit(df_ml[cols_num+cols_cat])


In [0]:
%python

# Obtener puntajes de anomalía
df_model_total['anomaly_score_isolation'] = best_model_iso.decision_function(df_ml[cols_num+cols_cat])
df_model_total['is_anomaly_isolation'] = best_model_iso.predict(df_ml[cols_num+cols_cat])  # -1 = Anomalía, 1 = Normal

# Ordenar por anomalía (los valores más bajos son más sospechosos)
df_sorted = df_model_total.sort_values(by='anomaly_score_isolation')

# Mostrar las compras más sospechosas
print(df_sorted.head(10))  # Top 10 compras más anómalas

In [0]:
%python
display(df_model_total)

#### DBSCAN

In [0]:
%python
#!pip install umap-learn
!pip uninstall umap-learn

In [0]:
%python
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import umap.umap_ as umap
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer

In [0]:
%python
# Aplicar reducción de dimensionalidad con UMAP
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=5, random_state=42)
X_umap = umap_model.fit_transform(df_ml[cols_cat+cols_num])

In [0]:
%python
X_umap.shape

In [0]:
%python
# Estimación de épsilon óptimo usando el método de rodilla
nearest_neighbors = NearestNeighbors(n_neighbors=12)
nearest_neighbors.fit(X_umap)
distances, _ = nearest_neighbors.kneighbors(X_umap)
distances = np.sort(distances[:, -1])

plt.plot(distances)
plt.xlabel("Puntos ordenados")
plt.ylabel("Distancia al 5º vecino más cercano")
plt.title("Curva de rodilla para seleccionar épsilon")
plt.show()

In [0]:
%python
# Entrenar DBSCAN con valores iniciales de épsilon y min_samples
eps_optimo = 0.15  # Ajusta este valor con base en la curva de rodilla
min_samples_optimo = 10

dbscan = DBSCAN(eps=eps_optimo, min_samples=min_samples_optimo, metric='euclidean')
clusters = dbscan.fit_predict(df_ml[cols_cat+cols_num])

In [0]:
%python
#  Definir función de evaluación para DBSCAN
def evaluate_dbscan(eps, min_samples):
    """Evalúa DBSCAN con los parámetros dados y devuelve la Silhouette Score"""
    model = DBSCAN(eps=eps, min_samples=int(min_samples))
    labels = model.fit_predict(X_umap)

    # Evita evaluar si todos los puntos son ruido (-1)
    if len(set(labels)) <= 1:
        return -1  # Silhouette Score no es válido en este caso
    
    return silhouette_score(X_umap, labels)
  
#  Configurar la Búsqueda Bayesiana
search_space = {
    'eps': Real(0.1, 2.0, prior='log-uniform'),  # Rango típico para DBSCAN
    'min_samples': Integer(2, 50)  # Número mínimo de puntos en un cluster
}  

opt = BayesSearchCV(
    estimator=DBSCAN(),
    search_spaces=search_space,
    scoring=lambda estimator, X: evaluate_dbscan(estimator.eps, estimator.min_samples),
    n_iter=20,  # Número de iteraciones de búsqueda
    random_state=42,
    cv=3  # Número de validaciones cruzadas
)

opt.fit(X_umap)

# Obtener los mejores parámetros
best_params_dbs = opt.best_params_
print(f"Mejores parámetros encontrados: {best_params_dbs}")

In [0]:
%python
#Mejores parámetros encontrados: OrderedDict([('eps', 0.36891624267080253), ('min_samples', 34)])

# Entrenar DBSCAN con los mejores parámetros
dbscan = DBSCAN(eps=best_params_dbs['eps'], min_samples=best_params_dbs['min_samples'])
clusters_dbs = dbscan.fit_predict(X_umap)

# Agregar resultados al DataFrame
df_model_total['cluster_dbscan'] = clusters_dbs
df_model_total['anomaly_score'] = (df_model_total['cluster_dbscan'] == -1).astype(int)

# Resumen de anomalías detectadas
print(df_model_total['cluster_dbscan'].value_counts())

#### Clasificar Datos Anomalos  

In [0]:
%python
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, precision_recall_curve

In [0]:
%python
display(df_model_total)

In [0]:
%python
np.percentile(df_model_total['reconstruction_error_autoencoder'], 95)

In [0]:
%python
# Definir las columnas de puntajes de anomalía (ajusta según tu dataset)
anomaly_scores_cols = [
    'reconstruction_error_autoencoder',
    'anomaly_score_k_mean',
    'anomaly_score_isolation'
    #, 'anomaly_score_kmeans'
    ]

# Definir un umbral de referencia basado en el percentil 95 si no hay etiquetas
for col in anomaly_scores_cols:
    threshold_percentile = np.percentile(df_model_total[col], 95)  # Umbral basado en distribución
    df_model_total[f'{col}_anomaly'] = (df_model_total[col] >= threshold_percentile).astype(int)

# Si hay etiquetas de anomalía, usar ROC y el criterio de Youden
if 'label' in df_model_total.columns:  # Suponiendo que 'label' tiene 1=anómalo, 0=no anómalo
    optimal_thresholds = {}

    for col in anomaly_scores_cols:
        fpr, tpr, thresholds = roc_curve(df_model_total['label'], df_model_total[col])
        youden_index = tpr - fpr  # Se maximiza este índice
        best_threshold = thresholds[np.argmax(youden_index)]

        optimal_thresholds[col] = best_threshold

        # Crear columna de clasificación binaria con el umbral óptimo
        df_model_total[f'{col}_anomaly'] = (df_model_total[col] >= best_threshold).astype(int)

    print("Puntos de corte óptimos calculados con ROC-Youden:", optimal_thresholds)

In [0]:
%python
# Contar las ocurrencias de cada categoría
category_counts = df_model_total['reconstruction_error_autoencoder_anomaly'].value_counts()

# Crear el gráfico de barras
plt.figure(figsize=(12, 8))
bars = plt.bar(category_counts.index, category_counts.values)
plt.title('Reconstruction Error Autoencoder Anomaly', fontsize=20)
plt.xlabel('Categoría', fontsize=16)
plt.ylabel('Conteo', fontsize=16)
plt.xticks(category_counts.index, fontsize=14)
plt.yticks(fontsize=14)

# Añadir los números del conteo sobre las barras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.1, int(yval), ha='center', va='bottom', fontsize=14)

plt.show()


In [0]:
%python
# Contar las ocurrencias de cada categoría
category_counts = df_model_total['anomaly_score_k_mean_anomaly'].value_counts()

# Crear el gráfico de barras
plt.figure(figsize=(12, 8))
bars = plt.bar(category_counts.index, category_counts.values)
plt.title('anomaly_score_k_mean_anomaly', fontsize=20)
plt.xlabel('Categoría', fontsize=16)
plt.ylabel('Conteo', fontsize=16)
plt.xticks(category_counts.index, fontsize=14)
plt.yticks(fontsize=14)

# Añadir los números del conteo sobre las barras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.1, int(yval), ha='center', va='bottom', fontsize=14)

plt.show()


In [0]:
%python
# Contar las ocurrencias de cada categoría
category_counts = df_model_total['anomaly_score_isolation_anomaly'].value_counts()

# Crear el gráfico de barras
plt.figure(figsize=(12, 8))
bars = plt.bar(category_counts.index, category_counts.values)
plt.title('anomaly_score_isolation_anomaly', fontsize=20)
plt.xlabel('Categoría', fontsize=16)
plt.ylabel('Conteo', fontsize=16)
plt.xticks(category_counts.index, fontsize=14)
plt.yticks(fontsize=14)

# Añadir los números del conteo sobre las barras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.1, int(yval), ha='center', va='bottom', fontsize=14)

plt.show()
